# <h2 style="text-align:center">Data Encryption Standard </h2>

In [1]:
# Path: DES.ipynb
# 
# # DES
#   - Data Encryption Standard
#  - Block cipher
# - 64-bit block size
# - 56-bit key size
# - 16 rounds
# - 8 S-boxes
# - 1 initial permutation
# - 1 final permutation
# - 1 expansion permutation
# - 1 permutation
# - 1 key schedule
# 
# ## Key Schedule
# - 56-bit key
# - Apply PC1 permutation
# - Split into left and right halves
# - Generate 16 round keys
#  - Apply left shift
# - Join halves and apply PC2 permutation
# 
# ## Encryption
# - 64-bit plaintext
# - Apply initial permutation
# - Split into left and right halves
# - Perform 16 rounds
# - Expand right half
# - XOR with round key
# - S-boxes
# - Permutation
# - XOR with left half
# - Swap left and right halves
# - Join left and right halves and apply final permutation
# 
# ## Decryption
# - 64-bit ciphertext
# - Apply initial permutation
# - Split into left and right halves
# - Perform 16 rounds
# - Expand right half
# - XOR with round key
# - S-boxes
# - Permutation
# - XOR with left half
# - Swap left and right halves
# - Join left and right halves and apply final permutation
# 


In [3]:
import random

def encrypt(plaintext, key):
    nblock = ((len(plaintext) - 1) >> 3) + 1
    ciphertext = ''

    for block in nblock:
        value = 0
        pos = block << 3
        for i in range(8):
            value = (value)
            if pos + i < len(plaintext):
                value = value  | ord(plaintext[pos + i])

        des = DES( value, key)

        ctext = ''
        for i in range(8):
            letter = des & 0xff
            ctext = chr(letter) + ctext
            des = des >> 8
        ciphertext += ctext
    return ciphertext

def decrypt(ciphertext, key): 
    nblock = ((len(ciphertext) - 1) >> 3) + 1
    plaintext = ''

    for block in nblock:
        value = 0
        pos = block << 3
        for i in range(8):
            value = (value << 8)
            if pos + i < len(ciphertext):
                value = value | ord(ciphertext[pos + i])

        des = DES(value, key, True)

        ptext = ''
        for i in range(8):
            letter = des & 0xff
            ptext = chr(letter) + ptext
            des = des >> 8
        plaintext += ptext
    return plaintext    

def permulate(number, permutation, bits):
    result = 0
    for i in range(bits):
        result = (result << 1) | ((number >> (bits - permutation[i])) & 1)
    return result

def rotleft(key, bits):
    return ((key << bits) | (key >> (28 - bits))) & 0xfffffff

def rotright(key, bits):
    return ((key >> bits) | (key << (28 - bits))) & 0xfffffff

def sboxFun(number, Fun):
    row = ((number & 0x20) >> 4) | (number & 1)
    col = (number & 0x1e) >> 1
    return Fun[row][col]


def DES(number, key, decrypt=False):
    firstPerm = [57, 49, 41, 33, 25, 17, 9, 1,
                 60, 52, 44, 36, 28, 20, 12, 4,
                 62, 54, 46, 38, 30, 22, 14, 6,
                 64, 56, 48, 40, 32, 24, 16, 8,
                 59, 51, 43, 35, 27, 19, 11, 3,
                 61, 53, 45, 37, 29, 21, 13, 5,
                 63, 55, 47, 39, 31, 23, 15, 7,]
    expansionFun = [31, 0, 1, 2, 3, 4,
                    3, 4, 5, 6, 7, 8,
                    7, 8, 9, 10, 11, 12,
                    11, 12, 13, 14, 15, 16,
                    15, 16, 17, 18, 19, 20,
                    19, 20, 21, 22, 23, 24,
                    23, 24, 25, 26, 27, 28,
                    27, 28, 29, 30, 31, 0]
    lastPerm = [39, 7, 47, 15, 55, 23, 63, 31,
                38, 6, 46, 14, 54, 22, 62, 30,
                37, 5, 45, 13, 53, 21, 61, 29,
                36, 4, 44, 12, 52, 20, 60, 28,
                35, 3, 43, 11, 51, 19, 59, 27,
                34, 2, 42, 10, 50, 18, 58, 26,
                33, 1, 41, 9, 49, 17, 57, 25,
                32, 0, 40, 8, 48, 16, 56, 24]
    
    pc2 = [13, 16, 10, 23, 0, 4,
            2, 27, 14, 5, 20, 9,
           22, 18, 11, 3, 25, 7,
           15, 6, 26, 19, 12, 1,
           40, 51, 30, 36, 46, 54,
           29, 39, 50, 44, 32, 47,
           43, 48, 38, 55, 33, 52,
           45, 41, 49, 35, 28, 31]
    interPerm = [15, 6, 19, 20, 28, 11, 27, 16,
                 0, 14, 22, 25, 4, 17, 30, 9,
                 1, 7, 23, 13, 31, 26, 2, 8,
                 18, 12, 29, 5, 21, 10, 3, 24]
    
    sboxFun1 = [[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
                [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
                [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
                [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]]
    
    number = permulate(number, firstPerm, 64)

    L = number >> 32
    R = number & 0xffffffff

    if decrypt:
        for i in range(16 + 1): 
            key = rotleft(key, 56)
    
    for round in range(16):
        if not decrypt:
            key = rotleft(key, 56)
        else:
            key = rotright(key, 56)
        roundKey = permulate(key, pc2, 56)

        expansion = permulate(R, expansionFun, 32)

        intnum = expansion ^ roundKey

        snum = 0
        for nbox in range(8):
            chunk = (intnum >> (42 - nbox * 6)) & 0x3f
            sbox = sboxFun(chunk, sboxFun1)
            snum = (snum << 4) | sbox
        
        inter = permulate(snum, interPerm, 32)

        new_R = L ^ inter

        L = R
        R = new_R

    number = (R << 32) | L

    number = permulate(number, lastPerm, 64)

    return number

def printSTR(string):
    ptext = ''
    for char in string:
        if ord(char) < 32 or ord(char) > 126:
            ptext += '.'
        else:
            ptext += char
    return ptext

def main():
    random.seed(0)
    print('DES implementation by Quoc Khanh')
    text = input('Enter plaintext: ')
    while text != 'quit':
        key = random.randint(0, 0xffffffff)
        print('Key: 0x{:x}'.format(key))
        ciphertext = encrypt(text, key)
        print('Ciphertext: 0x{:x}'.format(ciphertext))
        plaintext = decrypt(ciphertext, key)
        print('Plaintext: {}'.format(printSTR(plaintext)))
        text = input('Enter plaintext: ') 

if __name__ == '__main__':
    main()


DES implementation by Quoc Khanh


Key: 0xd82c07cd


TypeError: 'int' object is not iterable

# <p style="text-align:center">Author: Nguyen Quoc Khanh </p>